# Setup Enviorment

In [1]:
from Preprocessing.selfsupervised_data_praparation import plot_samples, get_dataset_list
from Preprocessing.generate_embeddings import generate_embeddings_df, save_embeddings_as_csv
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV2, VGG16, ResNet50V2

import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 

In [2]:
# Path to Dataset
path = '../DATASET_augmented/'
model_name = 'variational_autoencoder' #'VGG16' #'MobileNetV2' # 'vit' # 'autoencoder' #'ResNet50V2'
latent_dim = 1024

target_size = (224, 224, 3)
model_input = (224, 224, 3)
band = None

# Model path
model_path = f'Weights/{model_name}_{target_size[0]}_{latent_dim}.h5'  

if target_size[2] == 1:
    # Embeddings path
    embeddings_path = f'Embeddings_Augmented/{model_name}_Per_Band/{target_size[0]}_band{band+1}.csv'
else:
    # Embeddings path
    embeddings_path = f'Embeddings_Augmented/{model_name}_{target_size[0]}_{latent_dim}.csv'

## Read dataset

In [3]:
#plot_samples(path)

In [4]:
image_list = get_dataset_list(path, show_dirs=True, head=0)

Images in directories: 
54001
41001
5001
50001
68001
8001
23001
76001
73001
5360


## Load Model

In [5]:
tf.random.set_seed(0)

if model_name == 'variational_autoencoder':
    from Models import Variational_Autoencoder
    model = Variational_Autoencoder.get_Variational_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim, lr=0.0001)
elif model_name == 'autoencoder':
    from Models import Autoencoder
    model = Autoencoder.get_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim)
elif model_name == 'vit':
    from Models import ViT
    model = ViT.get_vit_backbone(model_input)
elif model_name == 'MobileNetV2':
    cnn = MobileNetV2(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'VGG16': # min depth
    cnn = VGG16(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'ResNet50V2':
    cnn = ResNet50V2(input_shape=model_input, include_top=False, weights='imagenet') 

if model_name in ['MobileNetV2', 'VGG16', 'ResNet50V2']:
    model = Sequential()
    model.add(cnn)
    model.add(tf.keras.layers.GlobalAveragePooling2D())

    # freeze:
    for layer in model.layers:
        layer.trainable = False
        
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


2023-03-11 20:01:53.116580: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-11 20:01:53.116622: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-11 20:01:53.116645: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (9e0b1a08816a): /proc/driver/nvidia/version does not exist
2023-03-11 20:02:09.430259: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 1024)              524389312 
Total params: 524,389,312
Trainable params: 0
Non-trainable params: 524,389,312
_________________________________________________________________


## Generate embeddings

In [6]:
if target_size[2] == 1:
    embeddings = generate_embeddings_df(image_list=image_list, model=model, target_size=target_size, BAND=band)
else:
    embeddings = generate_embeddings_df(image_list=image_list, model=model, target_size=target_size)
embeddings

,Municipality Code,Date,Embedding
0,54001,2017-07-30,"[-0.9347905, -0.7955346, -0.30496052, -1.15579..."
1,54001,2015-12-20,"[1.2887714, -0.24433734, 0.9235142, 1.1333696,..."
2,54001,2017-11-19,"[0.25444573, 0.11851785, -0.4407146, -2.044583..."
3,54001,2016-11-20,"[0.77081996, -0.05597853, 1.0672606, 0.525921,..."
4,54001,2017-07-16,"[0.43143496, -0.7728535, 0.50686777, 1.2077396..."
...,...,...,...
1634,5360,2017-04-16,"[0.72270215, -1.1955142, 0.39677137, 0.9343542..."
1635,5360,2018-12-02,"[-0.3338759, -0.6020818, -1.3969773, 1.0286756..."
1636,5360,2018-05-20,"[0.026333798, 1.0469539, 0.1553763, 0.6018746,..."
1637,5360,2016-09-18,"[1.547876, -0.70433515, 1.035635, -2.0318527, ..."


In [7]:
save_embeddings_as_csv(df=embeddings, path=embeddings_path)

In [8]:
pd.read_csv(embeddings_path)

,Municipality Code,Date,0,1,2,3,4,5,6,7,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,54001,2017-07-30,-0.934790,-0.795535,-0.304961,-1.155793,0.956842,2.205339,-0.405537,0.644230,...,-0.695482,0.749766,-1.177921,1.807502,1.460893,-1.042777,1.189294,-0.167375,0.269596,-1.675892
1,54001,2015-12-20,1.288771,-0.244337,0.923514,1.133370,1.117708,-0.973006,-0.864337,-0.287605,...,-2.067650,-0.527061,-0.387012,0.673278,0.027916,0.478157,0.120923,0.219167,1.096163,0.667632
2,54001,2017-11-19,0.254446,0.118518,-0.440715,-2.044583,0.540913,0.125979,-2.199053,-0.670573,...,-1.120360,-0.601033,-0.982430,0.040215,-0.337517,0.801029,-0.557588,0.570312,0.519818,0.732004
3,54001,2016-11-20,0.770820,-0.055979,1.067261,0.525921,0.820865,-0.597792,-0.044921,-0.330081,...,1.334792,-0.061039,1.709435,-0.371922,-0.490850,-0.936326,-0.764040,-0.160527,-0.845898,0.229857
4,54001,2017-07-16,0.431435,-0.772853,0.506868,1.207740,0.524962,0.185478,1.206795,-0.214588,...,-1.955850,0.630905,1.256421,0.140370,1.611541,-1.324836,0.618343,0.627013,0.444351,0.001027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634,5360,2017-04-16,0.722702,-1.195514,0.396771,0.934354,1.017317,0.751196,-2.799659,0.835193,...,0.822230,0.326982,-2.340190,-1.349107,1.291022,1.281428,-1.299247,1.907923,-1.364800,-0.796813
1635,5360,2018-12-02,-0.333876,-0.602082,-1.396977,1.028676,0.881274,2.016552,0.537925,-0.176316,...,-2.275485,0.903417,-0.793546,1.534254,-0.245847,0.412841,1.036080,-1.092132,-1.278182,-0.437367
1636,5360,2018-05-20,0.026334,1.046954,0.155376,0.601875,1.486039,-0.757626,1.253391,1.698332,...,0.208760,-0.418590,-1.018504,2.258470,0.585090,0.534298,-0.267414,-0.379041,-1.984524,-0.225875
1637,5360,2016-09-18,1.547876,-0.704335,1.035635,-2.031853,1.454877,-0.988927,1.207051,-1.171921,...,0.770256,0.568619,-0.116042,-1.409180,0.300541,0.650479,-0.139763,-0.572331,0.840257,1.217785
